In [22]:
from pathlib import Path
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langgraph.prebuilt import tools_condition
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model
import time
import io
from contextlib import redirect_stdout
from datetime import datetime
from testing.response_eval_tools import set_api_key_from_path, initialize_evaluation_llm, evaluate_model_outputs_from_paths
from pathlib import Path
import json
from typing import List, Dict, Any
import uuid


set_api_key_from_path(Path('./testing/gemini-key'))

################# [ Validate GPU Available ] #################
import torch

# assert(torch.cuda.is_available())

##############################################################

chroma_path = "./test_db"
collection_name = 'multi_proceedings_test'
embedding_model_name = 'all-MiniLM-L6-v2'

client = chromadb.PersistentClient(path=chroma_path)
embedding_function = SentenceTransformerEmbeddingFunction(model_name=embedding_model_name)
collection = client.get_or_create_collection(name=collection_name, embedding_function=embedding_function)

llm = initialize_evaluation_llm()
# llm = init_chat_model("llama3.2:3b-instruct-q8_0", model_provider="ollama")

class ChatHistoryManager:
    def __init__(self, max_history_length=10):
        self.sessions = {}
        self.max_history_length = max_history_length

    def get_or_create_session(self, session_id: str) -> List:
        """Get or create a new chat session"""
        if session_id not in self.sessions:
            self.sessions[session_id] = []
        return self.sessions[session_id]

    def add_message(self, session_id: str, message) -> None:
        """Add a message to the chat history"""
        history = self.get_or_create_session(session_id)
        history.append(message)

        if len(history) > self.max_history_length * 2:  # Keep pairs of messages
            self.sessions[session_id] = history[-self.max_history_length*2:]

    def get_history(self, session_id: str) -> List:
        """Get the chat history for a session"""
        return self.get_or_create_session(session_id)

    def clear_history(self, session_id: str) -> None:
        """Clear the chat history for a session"""
        self.sessions[session_id] = []

    def save_history(self): # TODO: STORE HISTORY TO FILE
        pass

    def load_history(self): # TODO: LOAD HISTORY FROM FILE
        pass

chat_manager = ChatHistoryManager(max_history_length=15)



def pre_filter_query(query: str):
    """
    Pre-filter queries to determine processing branch.
    You can easily modify this to add new classifications.
    """
    classification_prompt = f"""Classify this query into ONE category:

1. GRC_SPECIFIC - Any question about:
   - California utilities (PG&E, SCE, SDG&E, etc.)
   - Rate cases, revenue requirements, testimony, exhibits
   - CPUC proceedings, decisions, or regulations
   - Anything a regulatory analyst might need to know
   - Questions referencing previous conversation or context
   - ANY question where retrieved documents MIGHT be helpful

2. GRC_GENERAL - ONLY the most basic definitional questions like:
   - "What does GRC stand for?"
   - "What is a General Rate Case?"
   - "What does CPUC mean?"
   (Use ONLY when you're 100% certain no documents would help)

3. NON_GRC - ONLY for clearly off-topic requests like:
   - Recipes, games, jokes, personal advice
   - Non-utility topics (movies, sports, health)
   - Obvious chatbot misuse

 4. GRC_LONGFORM - It is absolutely clear this response needs an essay. A paragraph or two clearly wont do.

IMPORTANT: When in doubt, choose GRC_SPECIFIC. We want to help users with any utility-related questions.

Query: {query}

Respond with ONLY the category name (GRC_SPECIFIC, GRC_GENERAL, NON_GRC, or GRC_LONGFORM)."""

    try:
        # Create a simple message for classification
        classification_messages = [
            SystemMessage(content="You are a classifier for California GRC regulatory queries. Respond with only the category name."),
            HumanMessage(content=classification_prompt)
        ]

        response = llm.invoke(classification_messages)
        category = response.content.strip().upper()

        # Ensure the category exists in our configuration
        if category not in QUERY_BRANCHES:
            category = "GRC_SPECIFIC"  # Default fallback

        return category

    except Exception as e:
        # If classification fails, default to GRC_SPECIFIC
        print(f"Pre-filter error: {e}")
        return "GRC_SPECIFIC"

# Set up retrieval tool
@tool(response_format="content_and_artifact")
def retrieve(query: str, k: int = 8):
    """Retrieve information related to a query."""
    # Query ChromaDB directly
    results = collection.query(
        query_texts=[query],
        n_results=k,
    )

    # Format results for LangChain compatibility
    retrieved_docs = []
    for i in range(len(results['ids'][0])):
        doc_id = results['ids'][0][i]
        content = results['documents'][0][i]
        metadata = results['metadatas'][0][i] if results['metadatas'][0] else {}

        doc = Document(page_content=content, metadata=metadata)
        retrieved_docs.append(doc)

    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

def create_branch_retrieval_node(branch_name: str):
    """Create a retrieval node for a specific branch."""
    def branch_retrieval(state: MessagesState):
        branch_config = QUERY_BRANCHES[branch_name]

        if not branch_config["has_retrieval"]:
            # Skip retrieval for this branch
            return {"messages": state["messages"]}

        # Get the latest human message
        latest_human_message = None
        for message in reversed(state["messages"]):
            if message.type == "human":
                latest_human_message = message
                break

        if latest_human_message is None:
            return {"messages": state["messages"]}

        # Create a tool call for retrieval with branch specific k value
        tool_call_id = str(uuid.uuid4())
        retrieval_message = AIMessage(
            content=f"I'll search for relevant information to answer your {branch_name.lower().replace('_', ' ')} question.",
            tool_calls=[{
                "name": "retrieve",
                "id": tool_call_id,
                "args": {"query": latest_human_message.content, "k": branch_config["retrieval_k"]}
            }]
        )

        return {
            "messages": state["messages"] + [retrieval_message]
        }

    return branch_retrieval

def create_branch_generate_node(branch_name: str):
    """Create a generate node for a specific branch."""
    def branch_generate(state: MessagesState):
        branch_config = QUERY_BRANCHES[branch_name]

        # Get filter message
        if branch_config["filter_message"]:
            response_message = AIMessage(content=branch_config["filter_message"])
            return {"messages": [response_message]}

        # Get system prompt for branch
        system_prompt = branch_config["system_prompt"]

        # If this branch has retrieval, get the retrieved documents
        if branch_config["has_retrieval"]:
            # Get generated ToolMessages
            recent_tool_messages = []
            for message in reversed(state["messages"]):
                if message.type == "tool":
                    recent_tool_messages.append(message)
                else:
                    break
            tool_messages = recent_tool_messages[::-1]

            # Format document context if available
            docs_content = "\n\n".join(doc.content for doc in tool_messages)
            system_prompt = system_prompt.format(document_context=f"Document Context: {docs_content}")
        else:
            # No document context for non-retrieval branches
            system_prompt = system_prompt.format(document_context="")

        # Get conversation messages excluding tool messages and tool-calling AI messages
        conversation_messages = []
        for message in state["messages"]:
            if message.type == "human":
                conversation_messages.append(message)
            elif message.type == "ai" and not getattr(message, "tool_calls", None):
                conversation_messages.append(message)

        prompt = [SystemMessage(content=system_prompt)] + conversation_messages

        # Run llm
        response = llm.invoke(prompt)
        return {"messages": [response]}

    return branch_generate


QUERY_BRANCHES = {
    "NON_GRC": {
        "has_retrieval": False,
        "retrieval_k": 0,
        "filter_message": ("I'm specifically designed to assist with California General Rate Case (GRC) proceedings and CPUC regulatory matters. "
                           "How can I help you with GRC-related questions, rate case analysis, or utility regulatory issues?"),
        "system_prompt": """You are a GRC specialist. Politely redirect non-GRC queries back to GRC topics."""
    },
    "GRC_GENERAL": {
        "has_retrieval": False,
        "retrieval_k": 0,
        "filter_message": None,
        "system_prompt": """<SYSTEM>
You are "GRC Regulatory Analysis Expert," an AI assistant specialized in California GRC proceedings.
</SYSTEM>
<IDENTITY AND EXPERTISE>
You are a regulatory specialist focused exclusively on California General Rate Case (GRC) proceedings and related CPUC filings.
</IDENTITY AND EXPERTISE>
<RESPONSE FORMAT>
Provide clear, concise definitions and explanations for basic GRC concepts. Use markdown formatting for readability.
</RESPONSE FORMAT>
<SCOPE LIMITATIONS>
Address only topics related to California GRC proceedings and CPUC regulatory matters.
</SCOPE LIMITATIONS>
Always end responses with: "Would you like me to explore any aspect of this response in greater depth or address related regulatory considerations?"
"""
    },
    "GRC_SPECIFIC": {
        "has_retrieval": True,
        "retrieval_k": 8,
        "filter_message": None,
        "system_prompt": """<SYSTEM>
You are "GRC Regulatory Analysis Expert," an AI assistant specialized in California GRC proceedings.
</SYSTEM>

<INFORMATION SOURCES>
Base your responses EXCLUSIVELY on:
1. Retrieved documents (HIGHEST PRIORITY)
2. User-provided context in the current session

The retrieval system has already provided you with the most relevant information.
Always cite your sources with specific references (e.g., "PG&E 2023 GRC, Exhibit 4, p.15").
Always link to the original document, it should be provided as part of the context.
</INFORMATION SOURCES>

<IDENTITY AND EXPERTISE>
You are a regulatory specialist focused exclusively on California General Rate Case (GRC) proceedings and related CPUC filings with expertise in:
- Rate case applications and testimony
- Revenue requirement analysis
- Procedural requirements and timelines
- CPUC decisions and precedents
</IDENTITY AND EXPERTISE>

<RESPONSE FORMAT>
Structure your responses with:
1. Concise summary of key findings
2. Detailed analysis with multiple supporting citations
3. Relevant regulatory background and historical context
4. Discussion of practical implications
5. Complete citations formatted as markdown links

Use markdown formatting (headers, tables, bullets) to enhance readability.
</RESPONSE FORMAT>

<PROFESSIONAL TONE>
Maintain a voice that is:
- Authoritative yet accessible
- Technically precise
- Thorough and explanatory
- Objective in regulatory interpretation
</PROFESSIONAL TONE>

<ACCURACY REQUIREMENTS>
- Never invent citations, docket numbers, or proceedings
- Clearly indicate when information is missing or insufficient
- Present multiple interpretations when guidance is ambiguous
- Quote directly from sources for critical regulatory language
</ACCURACY REQUIREMENTS>

<SCOPE LIMITATIONS>
Address only topics related to California GRC proceedings and CPUC regulatory matters.
For other topics, politely explain they fall outside your expertise.
</SCOPE LIMITATIONS>

Always end responses with: "Would you like me to explore any aspect of this response in greater depth or address related regulatory considerations?"

{document_context}
""" # document_context gets replaced by actual documents in retrieval
    },
    "GRC_LONGFORM": {
        "has_retrieval": False,
        "retrieval_k": 10,
        "filter_message": None,
        "system_prompt": None
    }
}

def execute_longform(state: MessagesState) -> Dict[str, Any]:
    """
    Run longform generation
    """
    print("Executing self-contained GRC_LONGFORM branch...")

    # Get config
    branch_config = QUERY_BRANCHES["GRC_LONGFORM"]
    latest_human_message = next((m for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), None)

    if not latest_human_message:
        # Fallback if no human message is found
        return {"messages": [AIMessage(content="I'm sorry, I couldn't find your query.")]}

    query = latest_human_message.content
    retrieval_k = branch_config["retrieval_k"]

    # EXECUTE LONGFORM

    print("Generating final essay response...")
    # Run llm
    response = llm.invoke(prompt)
    print("Generation complete.")

    return {"messages": [response]}

# Build the graph
def build_graph():
    graph_builder = StateGraph(MessagesState)

    # Node to classify the user's query
    def classifier_node(state: MessagesState):
        """Classify the query and add classification to state metadata."""
        latest_human_message = next((m for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), None)

        if not latest_human_message:
            return {"messages": state["messages"], "query_classification": "GRC_SPECIFIC"}

        category = pre_filter_query(latest_human_message.content)
        return {"messages": state["messages"], "query_classification": category}

    def route_to_branch(state: MessagesState):
        classification = state.get("query_classification", "GRC_SPECIFIC")
        if classification == "GRC_LONGFORM":
            # Just run longform response
            return "execute_longform"
        else:
            # Go to standard retrieval nodes for other branches
            return f"retrieve_{classification.lower()}"

    # Set up the graph connections
    graph_builder.add_node("classifier", classifier_node)
    graph_builder.add_node("tools", ToolNode([retrieve]))
    graph_builder.add_node("execute_longform", execute_longform)

    # Dynamically create and connect nodes for all standard branches
    for branch_name, config in QUERY_BRANCHES.items():
        if branch_name == "GRC_LONGFORM": # Skip the custom branch in this loop
            continue

        retrieval_node_name = f"retrieve_{branch_name.lower()}"
        graph_builder.add_node(retrieval_node_name, create_branch_retrieval_node(branch_name))

        generate_node_name = f"generate_{branch_name.lower()}"
        graph_builder.add_node(generate_node_name, create_branch_generate_node(branch_name))

        # Connect edges for standard branches
        if config["has_retrieval"]:
            # Path for branches that use the ToolNode
            graph_builder.add_edge(retrieval_node_name, "tools")
            graph_builder.add_edge("tools", generate_node_name)
        else:
            # Path for branches that do not use tools
            graph_builder.add_edge(retrieval_node_name, generate_node_name)

        graph_builder.add_edge(generate_node_name, END)

    graph_builder.set_entry_point("classifier")
    graph_builder.add_conditional_edges(
        "classifier",
        route_to_branch,
    )

    graph_builder.add_edge("execute_longform", END)

    return graph_builder.compile()

# Initialize the graph
graph = build_graph()

def process_query(query: str, session_id: str, retrieval_k: int = 8, enable_prefilter: bool = True) -> Dict[str, Any]:
    """
    Process a query with dynamic branch routing.

    Args:
        query: The user's query
        session_id: Session identifier for chat history
        retrieval_k: Number of documents to retrieve (can be overridden by branch config)
        enable_prefilter: Whether to apply pre-filtering (default: True)
    """
    start_time = time.time()

    # Get chat history
    history = chat_manager.get_history(session_id)

    # Format messages
    messages = []
    for msg in history:
        if msg["role"] == "user":
            messages.append(HumanMessage(content=msg["content"]))
        elif msg["role"] == "assistant":
            messages.append(AIMessage(content=msg["content"]))
        elif msg["role"] == "tool":
            # Create a tool message with appropriate attributes and a tool_call_id
            messages.append(ToolMessage(
                content=msg["content"],
                name=msg.get("tool_name", "retrieve"),
                tool_call_id=msg.get("tool_call_id", str(uuid.uuid4()))
            ))

    # Add the current query
    messages.append(HumanMessage(content=query))

    # Run the graph
    result = None
    tool_outputs = []
    query_classification = None

    # Process the query through the graph
    with io.StringIO() as buf, redirect_stdout(buf):
        for step in graph.stream(
            {"messages": messages},
            stream_mode="values",
        ):
            # Get classification if available
            if "query_classification" in step:
                query_classification = step["query_classification"]

            last_message = step["messages"][-1]
            if last_message.type == "tool":
                tool_outputs.append({
                    "tool_name": getattr(last_message, "name", "retrieve"),
                    "content": last_message.content
                })
            if last_message.type == "ai" and not getattr(last_message, "tool_calls", None):
                result = last_message.content

        debug_output = buf.getvalue()

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Add messages to chat history (excluding classification messages)
    chat_manager.add_message(session_id, {"role": "user", "content": query})

    # Add tool messages to history
    for tool_output in tool_outputs:
        chat_manager.add_message(session_id, {
            "role": "tool",
            "content": tool_output["content"],
            "tool_name": tool_output.get("tool_name", "retrieve")
        })

    if result:
        chat_manager.add_message(session_id, {"role": "assistant", "content": result})

    response = {
        "result": result,
        "processing_time": elapsed_time,
        "tool_outputs": tool_outputs,
        "session_id": session_id,
        "timestamp": datetime.now().isoformat(),
        "debug_output": debug_output if debug_output else None,
        "query_classification": query_classification,
        "branch_used": query_classification,
        "filtered_out": query_classification == "NON_GRC"
    }

    return response

def get_available_branches():
    """Get list of all available query branches."""
    return list(QUERY_BRANCHES.keys())

def get_chat_history(session_id: str):
    return chat_manager.get_history(session_id)

def clear_chat_history(session_id: str):
    chat_manager.clear_history(session_id)
    return {"status": "success", "message": f"Chat history cleared for session {session_id}"}

def generate_session_id():
    return str(uuid.uuid4())

In [23]:
class LLMChatSession():
    def __init__(self, console_mode = False) -> None:
        self.session_id = generate_session_id()
        self.console_mode = console_mode
        self.timestamp_queue = [] # New ones get inserted at end (using append); remove by deleting element 0
        self.max_queries = 15
        # When query, update message queue; if still at limit then stop, otherwise
    def query(self, user_input: str, k = None) -> Dict[str, Any]:
        # if self.is_under_limit():
        if k is None:
            response = process_query(user_input, self.session_id)
        else:
            response = process_query(user_input, self.session_id, k)


        if self.console_mode:
            print(f"\nGRC Assistant: {response['result']}")
            print(f"\nProcessing time: {response['processing_time']:.2f} seconds")
        self.timestamp_queue.append(datetime.fromisoformat(response['timestamp']))
        self.timestamp_queue.append(datetime.fromisoformat(response['timestamp']))

        return {'result': response['result'],
                'messages_remaining': self.max_queries - sum([(datetime.now() - timestamp).seconds < 60 for timestamp in self.timestamp_queue]),
                'sec_remaining': [60 - (datetime.now() - timestamp).seconds for timestamp in self.timestamp_queue],
                'tool_outputs': response['tool_outputs'],
                }
        # else:
        #     raise ConnectionRefusedError(f"Rate Limit Exceeded | Try again in {60 - (datetime.now() - self.timestamp_queue[0]).seconds} seconds")

    def is_under_limit(self) -> bool:
        while len(self.timestamp_queue) > 0:
            if (datetime.now() - self.timestamp_queue[0]).seconds > 60: # Is over 60 sec time limit
                del self.timestamp_queue[0]
                continue
            else:
                break
        return len(self.timestamp_queue) < 15

    def hist_free_query(self, user_input: str) -> Dict[str, Any]:
        clear_chat_history(self.session_id)
        out = self.query(user_input)
        clear_chat_history(self.session_id)
        return out

In [24]:
llm_session = LLMChatSession(console_mode=True)

In [25]:
llm_session.query("Hey, how's it going?")


GRC Assistant: I'm specifically designed to assist with California General Rate Case (GRC) proceedings and CPUC regulatory matters. How can I help you with GRC-related questions, rate case analysis, or utility regulatory issues?

Processing time: 0.61 seconds


{'result': "I'm specifically designed to assist with California General Rate Case (GRC) proceedings and CPUC regulatory matters. How can I help you with GRC-related questions, rate case analysis, or utility regulatory issues?",
 'messages_remaining': 13,
 'sec_remaining': [60, 60],
 'tool_outputs': []}

In [26]:
llm_session.query("What is a GRC?")


GRC Assistant: A **General Rate Case (GRC)** is a comprehensive regulatory proceeding before the California Public Utilities Commission (CPUC) in which a utility (like an energy company or water provider) requests to adjust its rates for services provided to customers.

*   **Purpose:** To determine the fair and reasonable rates that customers will pay for utility services over a multi-year period (typically three years).
*   **Scope:** A GRC examines all aspects of a utility's operations, including:
    *   Operating expenses (e.g., salaries, maintenance)
    *   Capital investments (e.g., infrastructure upgrades)
    *   Sales forecasts
    *   Cost of capital
    *   Overall financial health
*   **Process:** The GRC involves:
    *   The utility filing an application with the CPUC.
    *   CPUC staff and other parties (intervenors) reviewing the application and challenging the utility's proposals.
    *   Evidentiary hearings where parties present testimony and evidence.
    *   Th

{'result': "A **General Rate Case (GRC)** is a comprehensive regulatory proceeding before the California Public Utilities Commission (CPUC) in which a utility (like an energy company or water provider) requests to adjust its rates for services provided to customers.\n\n*   **Purpose:** To determine the fair and reasonable rates that customers will pay for utility services over a multi-year period (typically three years).\n*   **Scope:** A GRC examines all aspects of a utility's operations, including:\n    *   Operating expenses (e.g., salaries, maintenance)\n    *   Capital investments (e.g., infrastructure upgrades)\n    *   Sales forecasts\n    *   Cost of capital\n    *   Overall financial health\n*   **Process:** The GRC involves:\n    *   The utility filing an application with the CPUC.\n    *   CPUC staff and other parties (intervenors) reviewing the application and challenging the utility's proposals.\n    *   Evidentiary hearings where parties present testimony and evidence.\n 

In [ ]:
llm_session.query("What is a GRC?")

In [29]:
result = llm_session.query("What information do you vae on PG&E from your retrival")


GRC Assistant: Based on the retrieved documents related to Proceeding A.21-06-021, here's an overview of information concerning PG&E:

*   **GRC Application:** PG&E filed an application for its General Rate Case (GRC) to adjust its rates for services provided to customers ([https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M389/K956/389956574.PDF](https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M389/K956/389956574.PDF)).
*   **Revenue and Rate Changes:** PG&E proposed revenue, rate changes, and ratemaking mechanisms in its amended application ([https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M458/K799/458799427.PDF](https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M458/K799/458799427.PDF), [https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M454/K983/454983908.PDF](https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M454/K983/454983908.PDF)).
*   **Testimony and Readiness:** PG&E's case is based on witness testimony and accompanying attachments supporting its revenue request ([https

In [14]:
result

{'result': "I can provide information about PG&E (Pacific Gas and Electric Company) based on the retrieved documents, focusing on its General Rate Case (GRC) filings, compliance, and other regulatory matters. Here's a summary of the key areas I can address:\n\n1.  **GRC Filings and Updates:**\n    *   PG&E's readiness to proceed with its Track 2 request based on witness testimony and supporting attachments related to its revenue requirement request (PG&E A2106021).\n    *   PG&E's commitment to publish notices regarding its filings in newspapers and customer bills (PG&E A2106021).\n    *   PG&E submitted additional testimony to update non-labor escalation rates and adjust its GRC request to reflect tax changes (PG&E A2106021).\n\n2.  **Compliance and Regulatory Strategy:**\n    *   PG&E's Compliance and Regulatory Strategy forecast supports functions like regulatory strategy, customer experience, tariff interpretation, risk, compliance, and audit (PG&E A2106021).\n\n3.  **Customer Noti

In [27]:
result = llm_session.query("Based on the public record in proceeding A.21-06-021, analyze the differences between PG&E's original undergrounding proposal and what was authorized in Decision 23-11-069. Include: (1) comparison of the miles of undergrounding requested by PG&E versus what was approved by the CPUC and the stated rationale for the reduction in the decision, (2) analysis of the Administrative Law Judge's Proposed Decision versus the Alternate Proposed Decision regarding undergrounding miles and which approach was ultimately adopted, (3) summary of intervenor positions on undergrounding from their filed testimony, including any opposition or alternative proposals, (4) identification of the specific CPUC findings regarding cost-effectiveness of undergrounding versus covered conductor alternatives as stated in the final decision, and (5) explanation of how the decision addresses PG&E's 10,000-mile undergrounding program timeline based on the proceeding record.")


GRC Assistant: Here's an analysis of PG&E's undergrounding proposal in A.21-06-021, comparing the initial proposal to the final authorized plan in D.23-11-069, and addressing the points you raised.

### 1. Comparison of Undergrounding Miles: PG&E Request vs. CPUC Approval

*   **PG&E's Original Proposal:** PG&E initially proposed to underground approximately **3,300 miles** from 2023 to 2026, at a forecast cost of approximately **$9,980 million** (Source: {'chunk_index': 815, 'document_id': '499888041', 'proceeding_id': 'A2106021', 'source_url': 'https://docs.cpuc.ca.gov/PublishedDocs/Efile/G000/M499/K888/499888041.PDF'}).
*   **PG&E's Modified Proposal:** PG&E later modified its proposal in its Reply Brief, decreasing the proposed underground mileage in 2023-2026 from approximately 3,300 miles to **2,100 miles**, reducing forecast expenditures by $4.05 billion to a new requested figure of **$6.13 billion** (Source: {'chunk_index': 1, 'document_id': '501533409', 'proceeding_id': 'A210

In [28]:
get_chat_history(llm_session.session_id)

[{'role': 'user', 'content': "Hey, how's it going?"},
 {'role': 'assistant',
  'content': "I'm specifically designed to assist with California General Rate Case (GRC) proceedings and CPUC regulatory matters. How can I help you with GRC-related questions, rate case analysis, or utility regulatory issues?"},
 {'role': 'user', 'content': 'What is a GRC?'},
 {'role': 'assistant',
  'content': "A **General Rate Case (GRC)** is a comprehensive regulatory proceeding before the California Public Utilities Commission (CPUC) in which a utility (like an energy company or water provider) requests to adjust its rates for services provided to customers.\n\n*   **Purpose:** To determine the fair and reasonable rates that customers will pay for utility services over a multi-year period (typically three years).\n*   **Scope:** A GRC examines all aspects of a utility's operations, including:\n    *   Operating expenses (e.g., salaries, maintenance)\n    *   Capital investments (e.g., infrastructure upgr